In [9]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error
from keras.optimizers import Adam
from kerastuner.tuners import RandomSearch

<ipython-input-9-2a3157c3ee9f>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


#Testing LSTM on 1 dataset file - PRACTICE ONLY

In [5]:
data_df = pd.read_csv('/content/User_161.csv')


data = data_df.to_numpy()


max_vals = np.max(data, axis=0)
data_normalized = data / max_vals


sequence_length = 3

X, y = [], []
for i in range(len(data_normalized) - sequence_length):
    X.append(data_normalized[i:i+sequence_length])
    y.append(data_normalized[i+sequence_length])

X = np.array(X)
y = np.array(y)


split_ratio = 0.8
split_index = int(len(X) * split_ratio)

X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]


# Define hyperparameters
lstm_units = 128
learning_rate = 0.001
num_epochs = 200
batch_size = 64

# Create the model with hyperparameters
model = Sequential()
model.add(LSTM(units=lstm_units, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(2))
optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Train the model with hyperparameters
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size)



score = model.evaluate(X_test, y_test)
print("Test loss:", score)
input_data = data_normalized[-sequence_length:]
predicted_normalized = model.predict(np.array([input_data]))
predicted = predicted_normalized * max_vals


print("Predicted Cycle Length:", predicted[0][0].round())
print("Predicted Ovulation Day:", predicted[0][1].round())



# Calculate predictions for the test set
y_pred_normalized = model.predict(X_test)
y_pred = y_pred_normalized * max_vals

# Calculate Mean Absolute Error (MAE) and Mean Squared Error (MSE)
mae = mean_absolute_error(y_test * max_vals, y_pred)
mse = mean_squared_error(y_test * max_vals, y_pred)

print("Mean Absolute Error (MAE):", mae.round())
print("Mean Squared Error (MSE):", mse.round())

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 0.4964
Epoch 2/200
1/1 [==============================] - 0s 16ms/step - loss: 0.4791
Epoch 3/200
1/1 [==============================] - 0s 13ms/step - loss: 0.4621
Epoch 4/200
1/1 [==============================] - 0s 14ms/step - loss: 0.4457
Epoch 5/200
1/1 [==============================] - 0s 15ms/step - loss: 0.4295
Epoch 6/200
1/1 [==============================] - 0s 13ms/step - loss: 0.4136
Epoch 7/200
1/1 [==============================] - 0s 14ms/step - loss: 0.3979
Epoch 8/200
1/1 [==============================] - 0s 14ms/step - loss: 0.3824
Epoch 9/200
1/1 [==============================] - 0s 13ms/step - loss: 0.3670
Epoch 10/200
1/1 [==============================] - 0s 14ms/step - loss: 0.3520
Epoch 11/200
1/1 [==============================] - 0s 15ms/step - loss: 0.3372
Epoch 12/200
1/1 [==============================] - 0s 13ms/step - loss: 0.3227
Epoch 13/200
1/1 [==============================] -

# TESTING ON MULTIPLE DATASETS WITHOUT ANY HP TUNING

In [7]:
# Define hyperparameters
lstm_units = 128
learning_rate = 0.01
num_epochs = 200
batch_size = 128
sequence_length = 3



all_data_mae = 0
num_of_datasets_used = 0

dataset_files = [f'User_{i}.csv' for i in range(1, 162)]  # Generates filenames from User_1 to User_161

for dataset_file in dataset_files:
    # Load data from the current dataset file
    data_df = pd.read_csv('/content/' + dataset_file)
    data = data_df.to_numpy()
    if len(data) > 10:
      num_of_datasets_used += 1
      max_vals = np.max(data, axis=0)
      data_normalized = data / max_vals

      X, y = [], []
      for i in range(len(data_normalized) - sequence_length):
          X.append(data_normalized[i:i+sequence_length])
          y.append(data_normalized[i+sequence_length])

      X = np.array(X)
      y = np.array(y)

      split_ratio = 0.8
      split_index = int(len(X) * split_ratio)

      X_train, y_train = X[:split_index], y[:split_index]
      X_test, y_test = X[split_index:], y[split_index:]

      # Create the model with hyperparameters
      model = Sequential()
      model.add(LSTM(units=lstm_units, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
      model.add(Dense(2))
      optimizer = Adam(learning_rate=learning_rate)
      model.compile(loss='mean_squared_error', optimizer=optimizer)

      # Train the model with hyperparameters
      model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size)

      score = model.evaluate(X_test, y_test)
      print("Test loss for", dataset_file, ":", score)

      input_data = data_normalized[-sequence_length:]
      predicted_normalized = model.predict(np.array([input_data]))
      predicted = predicted_normalized * max_vals

      print("Predicted Cycle Length:", predicted[0][0].round())
      print("Predicted Ovulation Day:", predicted[0][1].round())

      # Calculate predictions for the test set
      y_pred_normalized = model.predict(X_test)
      y_pred = y_pred_normalized * max_vals

      # Calculate Mean Absolute Error (MAE) and Mean Squared Error (MSE)
      mae = mean_absolute_error(y_test * max_vals, y_pred)
      mse = mean_squared_error(y_test * max_vals, y_pred)

      print("Mean Absolute Error (MAE) for", dataset_file, ":", mae.round())
      print("Mean Squared Error (MSE) for", dataset_file, ":", mse.round())
      all_data_mae += mae
      print()
    else:
      print('Dataset size for ', dataset_file, ' is small!')
      print()

print()
all_data_mae_avg = all_data_mae / num_of_datasets_used
print('The avergae MAE for all the valid size datasets processed was: ', all_data_mae_avg)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 14ms/step - loss: 0.0012
Epoch 187/200
1/1 [==============================] - 0s 16ms/step - loss: 0.0012
Epoch 188/200
1/1 [==============================] - 0s 14ms/step - loss: 0.0012
Epoch 189/200
1/1 [==============================] - 0s 15ms/step - loss: 0.0012
Epoch 190/200
1/1 [==============================] - 0s 16ms/step - loss: 0.0012
Epoch 191/200
1/1 [==============================] - 0s 16ms/step - loss: 0.0012
Epoch 192/200
1/1 [==============================] - 0s 15ms/step - loss: 0.0012
Epoch 193/200
1/1 [==============================] - 0s 14ms/step - loss: 0.0012
Epoch 194/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0012
Epoch 195/200
1/1 [==============================] - 0s 14ms/step - loss: 0.0012
Epoch 196/200
1/1 [==============================] - 0s 16ms/step - loss: 0.0012
Epoch 197/200
1/1 [==============================] - 0s 23ms/step - loss

# KerasTuner: Hyperparameter Tuning (Random Search)

In [8]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [10]:
# Define hyperparameters
lstm_units = 128
learning_rate = 0.01
num_epochs = 200
batch_size = 128
sequence_length = 3

# Define a function to build the LSTM model with hyperparameters
def build_lstm_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units', min_value=32, max_value=256, step=32), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(2))
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.01, 0.1]))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

# Set up the RandomSearch tuner
tuner = RandomSearch(
    build_lstm_model,
    objective='val_loss',  # You can choose another objective based on your preference
    max_trials=20,  # Number of hyperparameter combinations to try
    directory='my_tuning_directory_1',  # Specify a directory for storing the results
    project_name='lstm_hyperparameter_tuning'
)

# Define an early stopping callback
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20)  # Adjust patience as needed

all_data_mae = 0
num_of_datasets_used = 0

dataset_files = [f'User_{i}.csv' for i in range(1, 162)]  # Adjust the range as needed

for dataset_file in dataset_files:
    # Load data from the current dataset file
    data_df = pd.read_csv('/content/' + dataset_file)
    data = data_df.to_numpy()
    if len(data) >= 8:
        num_of_datasets_used += 1
        max_vals = np.max(data, axis=0)
        data_normalized = data / max_vals

        X, y = [], []
        for i in range(len(data_normalized) - sequence_length):
            X.append(data_normalized[i:i+sequence_length])
            y.append(data_normalized[i+sequence_length])

        X = np.array(X)
        y = np.array(y)

        split_ratio = 0.8
        split_index = int(len(X) * split_ratio)

        X_train, y_train = X[:split_index], y[:split_index]
        X_test, y_test = X[split_index:], y[split_index:]

        # Perform hyperparameter tuning
        tuner.search(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[early_stopping])

        # Get the best model from the tuner
        best_model = tuner.get_best_models(1)[0]

        # Continue with evaluation and predictions using the best model
        score = best_model.evaluate(X_test, y_test)
        input_data = data_normalized[-sequence_length:]
        predicted_normalized = best_model.predict(np.array([input_data]))
        predicted = predicted_normalized * max_vals

        print("Predicted Cycle Length:", predicted[0][0].round())
        print("Predicted Ovulation Day:", predicted[0][1].round())

        # Calculate predictions for the test set
        y_pred_normalized = best_model.predict(X_test)
        y_pred = y_pred_normalized * max_vals

        # Calculate Mean Absolute Error (MAE) and Mean Squared Error (MSE)
        mae = mean_absolute_error(y_test * max_vals, y_pred)
        mse = mean_squared_error(y_test * max_vals, y_pred)

        print("Mean Absolute Error (MAE) for", dataset_file, ":", mae.round())
        print("Mean Squared Error (MSE) for", dataset_file, ":", mse.round())
        all_data_mae += mae
        print()
    else:
        print('Dataset size for ', dataset_file, ' is small!')
        print()
print()
all_data_mae_avg = all_data_mae / num_of_datasets_used
print('The avergae MAE for all the valid size datasets processed was: ', all_data_mae_avg)

Trial 20 Complete [00h 00m 24s]
val_loss: 0.0025596763007342815

Best val_loss So Far: 0.002201998606324196
Total elapsed time: 00h 06m 26s
1/1 [==============================] - 0s 198ms/step
Predicted Cycle Length: 27.0
Predicted Ovulation Day: 15.0
1/1 [==============================] - 0s 24ms/step
Mean Absolute Error (MAE) for User_1.csv : 1.0
Mean Squared Error (MSE) for User_1.csv : 1.0

Dataset size for  User_2.csv  is small!

Dataset size for  User_3.csv  is small!

Dataset size for  User_4.csv  is small!

Dataset size for  User_5.csv  is small!

1/1 [==============================] - 0s 230ms/step
Predicted Cycle Length: 28.0
Predicted Ovulation Day: 21.0
1/1 [==============================] - 0s 29ms/step
Mean Absolute Error (MAE) for User_6.csv : 3.0
Mean Squared Error (MSE) for User_6.csv : 17.0



1/1 [==============================] - 0s 292ms/step
Predicted Cycle Length: 26.0
Predicted Ovulation Day: 15.0
1/1 [==============================] - 0s 36ms/step
Mean Absolute Error (MAE) for User_7.csv : 1.0
Mean Squared Error (MSE) for User_7.csv : 4.0

Dataset size for  User_8.csv  is small!



1/1 [==============================] - 0s 201ms/step
Predicted Cycle Length: 25.0
Predicted Ovulation Day: 13.0
1/1 [==============================] - 0s 23ms/step
Mean Absolute Error (MAE) for User_9.csv : 3.0
Mean Squared Error (MSE) for User_9.csv : 7.0



1/1 [==============================] - 0s 205ms/step
Predicted Cycle Length: 24.0
Predicted Ovulation Day: 12.0
1/1 [==============================] - 0s 22ms/step
Mean Absolute Error (MAE) for User_10.csv : 2.0
Mean Squared Error (MSE) for User_10.csv : 4.0



1/1 [==============================] - 0s 207ms/step
Predicted Cycle Length: 35.0
Predicted Ovulation Day: 17.0
1/1 [==============================] - 0s 23ms/step
Mean Absolute Error (MAE) for User_11.csv : 3.0
Mean Squared Error (MSE) for User_11.csv : 13.0

Dataset size for  User_12.csv  is small!

Dataset size for  User_13.csv  is small!

Dataset size for  User_14.csv  is small!



...

1/1 [==============================] - 0s 202ms/step
Predicted Cycle Length: 41.0
Predicted Ovulation Day: 23.0
1/1 [==============================] - 0s 24ms/step
Mean Absolute Error (MAE) for User_157.csv : 7.0
Mean Squared Error (MSE) for User_157.csv : 48.0



1/1 [==============================] - 0s 202ms/step
Predicted Cycle Length: 32.0
Predicted Ovulation Day: 19.0
1/1 [==============================] - 0s 24ms/step
Mean Absolute Error (MAE) for User_158.csv : 3.0
Mean Squared Error (MSE) for User_158.csv : 12.0



1/1 [==============================] - 0s 312ms/step
Predicted Cycle Length: 42.0
Predicted Ovulation Day: 11.0
1/1 [==============================] - 0s 39ms/step
Mean Absolute Error (MAE) for User_159.csv : 4.0
Mean Squared Error (MSE) for User_159.csv : 22.0



1/1 [==============================] - 0s 212ms/step
Predicted Cycle Length: 30.0
Predicted Ovulation Day: 18.0
1/1 [==============================] - 0s 24ms/step
Mean Absolute Error (MAE) for User_160.csv : 1.0
Mean Squared Error (MSE) for User_160.csv : 3.0



1/1 [==============================] - 0s 205ms/step
Predicted Cycle Length: 33.0
Predicted Ovulation Day: 21.0
1/1 [==============================] - 0s 27ms/step
Mean Absolute Error (MAE) for User_161.csv : 6.0
Mean Squared Error (MSE) for User_161.csv : 35.0


The avergae MAE for all the valid size datasets processed was:  2.4165934110628933


# Hyperparamter Tuning - Grid Search CV

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error
from keras.optimizers import Adam
from kerastuner.tuners import GridSearch
from kerastuner.engine.hyperparameters import HyperParameters

# Define hyperparameters
lstm_units = 128
learning_rate = 0.1
num_epochs = 100
batch_size = 64
sequence_length = 3

# Define the hypermodel function
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int("lstm_units", min_value=32, max_value=256, step=32), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(2))
    optimizer = Adam(learning_rate=hp.Choice("learning_rate", values=[0.01, 0.1]))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

# Define the objective
objective = "val_loss"

all_data_mae = 0
num_of_datasets_used = 0

dataset_files = [f'User_{i}.csv' for i in range(1, 162)]

for dataset_file in dataset_files:
    data_df = pd.read_csv('/content/' + dataset_file)
    data = data_df.to_numpy()

    if len(data) >= 10:
        num_of_datasets_used += 1
        max_vals = np.max(data, axis=0)
        data_normalized = data / max_vals

        X, y = [], []
        for i in range(len(data_normalized) - sequence_length):
            X.append(data_normalized[i:i+sequence_length])
            y.append(data_normalized[i+sequence_length])

        X = np.array(X)
        y = np.array(y)

        split_ratio = 0.8
        split_index = int(len(X) * split_ratio)

        X_train, y_train = X[:split_index], y[:split_index]
        X_test, y_test = X[split_index:], y[split_index:]

        # Set up the GridSearch tuner
        tuner = GridSearch(
            hypermodel=build_model,
            objective=objective,
            max_trials=10,  # Adjust the number of trials as needed
            directory='my_gridsearch_dir_7',
            project_name='my_tuning_project'
        )

        # Run the hyperparameter tuning process
        tuner.search(
            X_train,
            y_train,
            epochs=num_epochs,
            batch_size=batch_size,
            validation_data=(X_test, y_test)
        )

        # Get the best hyperparameters and build the final model
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
        final_model = build_model(best_hps)

        # Train the final model with the best hyperparameters
        final_model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size)

        # Evaluate the final model and make predictions
        score = final_model.evaluate(X_test, y_test)
        print("Test loss for", dataset_file, ":", score)

        input_data = data_normalized[-sequence_length:]
        predicted_normalized = final_model.predict(np.array([input_data]))
        predicted = predicted_normalized * max_vals

        print("Predicted Cycle Length:", predicted[0][0].round())
        print("Predicted Ovulation Day:", predicted[0][1].round())

        # Calculate predictions for the test set
        y_pred_normalized = final_model.predict(X_test)
        y_pred = y_pred_normalized * max_vals

        # Calculate Mean Absolute Error (MAE) and Mean Squared Error (MSE)
        mae = mean_absolute_error(y_test * max_vals, y_pred)
        mse = mean_squared_error(y_test * max_vals, y_pred)


        print("Mean Absolute Error (MAE) for", dataset_file, ":", mae.round())
        print("Mean Squared Error (MSE) for", dataset_file, ":", mse.round())
        all_data_mae += mae
        print()
    else:
        print('Dataset size for ', dataset_file, ' is small!')
        print()

print()
all_data_mae_avg = all_data_mae / num_of_datasets_used
print('The average MAE for all the valid size datasets processed was: ', all_data_mae_avg)

# Save the final model to a file
final_model.save("best_model.h5")



Trial 10 Complete [00h 00m 07s]
val_loss: 0.0028577647171914577

Best val_loss So Far: 0.00212501036003232
Total elapsed time: 00h 01m 29s
Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.7938
Epoch 2/100
1/1 [==============================] - 0s 13ms/step - loss: 0.1850
Epoch 3/100
1/1 [==============================] - 0s 11ms/step - loss: 0.0912
Epoch 4/100
1/1 [==============================] - 0s 12ms/step - loss: 0.1437
Epoch 5/100
1/1 [==============================] - 0s 15ms/step - loss: 0.1030
Epoch 6/100
1/1 [==============================] - 0s 18ms/step - loss: 0.0440
Epoch 7/100
1/1 [==============================] - 0s 16ms/step - loss: 0.0180
Epoch 8/100
1/1 [==============================] - 0s 15ms/step - loss: 0.0445
Epoch 9/100
1/1 [==============================] - 0s 17ms/step - loss: 0.0548
Epoch 10/100
1/1 [==============================] - 0s 12ms/step - loss: 0.0285
Epoch 11/100
1/1 [==============================] - 0s 10ms/step - loss:

1/1 [==============================] - 0s 357ms/step - loss: 0.0136
Test loss for User_9.csv : 0.013646450825035572
1/1 [==============================] - 0s 253ms/step
Predicted Cycle Length: 27.0
Predicted Ovulation Day: 14.0
1/1 [==============================] - 0s 49ms/step
Mean Absolute Error (MAE) for User_9.csv : 2.0
Mean Squared Error (MSE) for User_9.csv : 6.0

Dataset size for  User_10.csv  is small!

Reloading Tuner from my_gridsearch_dir_7/my_tuning_project/tuner0.json
Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.6409
Epoch 2/100
1/1 [==============================] - 0s 11ms/step - loss: 1.8055
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 0.2307
Epoch 4/100
1/1 [==============================] - 0s 14ms/step - loss: 0.5275
Epoch 5/100
1/1 [==============================] - 0s 13ms/step - loss: 0.5702
Epoch 6/100
1/1 [==============================] - 0s 11ms/step - loss: 0.5385
Epoch 7/100
1/1 [=========================

1/1 [==============================] - 0s 229ms/step - loss: 0.0067
Test loss for User_11.csv : 0.006729624234139919


1/1 [==============================] - 0s 169ms/step
Predicted Cycle Length: 35.0
Predicted Ovulation Day: 19.0
1/1 [==============================] - 0s 20ms/step
Mean Absolute Error (MAE) for User_11.csv : 2.0
Mean Squared Error (MSE) for User_11.csv : 11.0

Dataset size for  User_12.csv  is small!

Dataset size for  User_13.csv  is small!

Dataset size for  User_14.csv  is small!

Reloading Tuner from my_gridsearch_dir_7/my_tuning_project/tuner0.json
Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 0.8275
Epoch 2/100
1/1 [==============================] - 0s 10ms/step - loss: 0.1929
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 0.2043
Epoch 4/100
1/1 [==============================] - 0s 8ms/step - loss: 0.2585
Epoch 5/100
1/1 [==============================] - 0s 11ms/step - loss: 0.2016
Epoch 6/100
1/1 [==============================] - 0s 10ms/step - loss: 0.1238
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: 

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 12ms/step - loss: 0.0070
Epoch 54/100
1/1 [==============================] - 0s 17ms/step - loss: 0.0070
Epoch 55/100
1/1 [==============================] - 0s 12ms/step - loss: 0.0070
Epoch 56/100
1/1 [==============================] - 0s 10ms/step - loss: 0.0070
Epoch 57/100
1/1 [==============================] - 0s 19ms/step - loss: 0.0070
Epoch 58/100
1/1 [==============================] - 0s 11ms/step - loss: 0.0070
Epoch 59/100
1/1 [==============================] - 0s 11ms/step - loss: 0.0069
Epoch 60/100
1/1 [==============================] - 0s 17ms/step - loss: 0.0069
Epoch 61/100
1/1 [==============================] - 0s 11ms/step - loss: 0.0069
Epoch 62/100
1/1 [==============================] - 0s 10ms/step - loss: 0.0068
Epoch 63/100
1/1 [==============================] - 0s 21ms/step - loss: 0.0068
Epoch 64/100
1/1 [==============================] - 0s 10ms/step - loss: 0.0068
Ep

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Loading the best model and making predictions

In [ ]:
import pandas as pd
import tensorflow as tf
model_new = tf.keras.models.load_model('/content/best_model.h5')

user_data_df = pd.read_csv('/content/User_150.csv')
user_data = user_data_df.to_numpy()

max_vals = np.max(user_data, axis=0)
user_input_normalized = user_data / max_vals

user_input_sequences = []
for i in range(len(user_input_normalized) - sequence_length):
    user_input_sequences.append(user_input_normalized[i:i+sequence_length])

user_input_sequences = np.array(user_input_sequences)


predicted_normalized = model_new.predict(np.array([user_input_sequences[-1]]))
predicted = predicted_normalized * max_vals

predicted_cycle_length = predicted[0][0].round()
predicted_ovulation_day = predicted[0][1].round()

print("Predicted Cycle Length:", predicted_cycle_length)
print("Predicted Ovulation Day:", predicted_ovulation_day)


1/1 [==============================] - 0s 166ms/step
Predicted Cycle Length: 25.0
Predicted Ovulation Day: 13.0
